In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-11-21 22:03:17--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.243, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-11-21 22:03:17--  http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)... 5.45.205.221, 2a02:6b8::3:221
Connecting to cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)|5.45.205.221|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.4’

mystem-3.0-linux3.1 100%[===================>]  15.70M 

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.feature_extraction.text import TfidfVectorizer


import re

from scipy.sparse import hstack, csr_matrix

from tqdm import tqdm_notebook
import matplotlib.pyplot as plt

import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ROOT = '/content/drive/MyDrive/ml/hacks/avito/data/'
image_embeddings = ROOT + 'images_mobilenetv22.csv' 
target = 'deal_probability'
category_columns = ['region', 'city', 'parent_category_name', 'category_name', 'param_1', 'param_2', 'param_3', 'user_type', 'item_seq_number']
numerical_columns = ['image_top_1', 'price']
text_columns = ['title', 'description']
meta_columns = ['item_id', 'user_id', 'image', 'activation_date']
all_columns = numerical_columns + category_columns + text_columns + meta_columns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)
pd.set_option('max_seq_item', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('precision', 10)

In [ ]:
train = pd.read_csv(ROOT + 'train.csv').sort_values('activation_date')
print(train.values.shape)
train.head()

(1503424, 18)


,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
751711,3b896605f03c,9457abfa0ebf,Самарская область,Самара,Личные вещи,Детская одежда и обувь,Для мальчиков,Верхняя одежда,86-92 см (1-2 года),Зимний комбенизон,Зимний комбенизон для малыша,500.0000,12,2017-03-15,Private,a805aa774699aca42e538e1626ef70f806dd38d1bf85ebfffbd039c895d157b3,1008.0000,0.0000
889369,c81c34d3aac5,0b161da2b144,Иркутская область,Братск,Бытовая электроника,Телефоны,Аксессуары,Чехлы и плёнки,NaN,Чехол для айфона 6s,Твёрдый пластик,100.0000,5,2017-03-15,Private,76c39c52a894c51499da3ee2ddecd98da533ccbdae5096ba48e7d1fe61108215,2909.0000,0.0000
175952,a8ab6225dd06,93d527a1ca66,Пермский край,Краснокамск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Верхняя одежда,44–46 (M),Продам кожанную куртку,NaN,1500.0000,32,2017-03-15,Private,d7cb3c4a84ce9f95958f2995474140b28ae7ad1cd0d6de5b8735eebf815c0572,527.0000,0.0000
889372,20bf5eea7fb5,29a0f14d92a9,Оренбургская область,Оренбург,Животные,Товары для животных,NaN,NaN,NaN,"Продам, игрушки + шлейку","Продам, цена указана за все что на фото!/\nВсе в идеальном состояние!/\nЕсли объявления на сайте значит актуально!",350.0000,17,2017-03-15,Private,b79c6bdfd4976a1d8acf1f78ae5df6f991e5b9dbf7bf0e126b115ed497ae3e8b,930.0000,0.0000
534674,e1cf6b9d5789,f555e70d49e6,Ярославская область,Ярославль,Личные вещи,"Одежда, обувь, аксессуары",Мужская одежда,Другое,NaN,Спортивный костюм adidas,"Спортивный костюм. Размер 48 - 50, мало б/у. Небольшой торг.",1500.0000,7,2017-03-15,Private,36a919a08374fba495be91342def93f9a19ed05660e6b40848445d9e7253a00c,657.0000,0.2731


In [ ]:
train.describe()

,price,item_seq_number,image_top_1,deal_probability
count,1418062.0000,1503424.0000,1390836.0000,1503424.0000
mean,316708.0885,743.6740,1241.9321,0.1391
std,66891542.1030,5572.5224,970.4641,0.2601
min,0.0000,1.0000,0.0000,0.0000
25%,500.0000,9.0000,425.0000,0.0000
50%,1300.0000,29.0000,1057.0000,0.0000
75%,7000.0000,88.0000,2217.0000,0.1509
max,79501011850.0000,204429.0000,3066.0000,1.0000


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1503424 entries, 751711 to 187627
Data columns (total 18 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   item_id               1503424 non-null  object 
 1   user_id               1503424 non-null  object 
 2   region                1503424 non-null  object 
 3   city                  1503424 non-null  object 
 4   parent_category_name  1503424 non-null  object 
 5   category_name         1503424 non-null  object 
 6   param_1               1441848 non-null  object 
 7   param_2               848882 non-null   object 
 8   param_3               640859 non-null   object 
 9   title                 1503424 non-null  object 
 10  description           1387148 non-null  object 
 11  price                 1418062 non-null  float64
 12  item_seq_number       1503424 non-null  int64  
 13  activation_date       1503424 non-null  object 
 14  user_type             1503424 

In [ ]:
for col in category_columns:
    train[col] = train[col].fillna('N/A').astype(str).str.lower()
    train.loc[train[col].value_counts()[train[col]].values < 100, col] = "rare"
    train = pd.concat([train, pd.get_dummies(train[col], prefix=col)], axis=1)

In [ ]:
train['price'] = np.log1p(train['price'].clip(0,5000000) + 1e-5)

agg_price = train[['parent_category_name','price']].groupby('parent_category_name')['price'].describe()

mean_parent_category_price = agg_price['mean'].reset_index()
mean_parent_category_price.columns = ['parent_category_name', 'mean_parent_category_price']
train = train.merge(mean_parent_category_price)
train['price'] = train['price'].fillna(train['mean_parent_category_price'])

std_parent_category_price = agg_price['std'].reset_index()
std_parent_category_price.columns = ['parent_category_name', 'std_parent_category_price']
train = train.merge(std_parent_category_price)

In [ ]:
train['image_top_1'] = train['image_top_1'].fillna(0)

In [ ]:
text_batch_size = 1000
def create_batches(lst, sz):
  return [lst[i:i+sz] for i in range(0, len(lst), sz)]

mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

for col in text_columns:
  train[col] = train[col].fillna(' ')
  series = train[col].tolist()
  batches = create_batches(series, text_batch_size)
  result = []
  for txtp in batches:
    concated = ' '.join([re.sub('([^а-я]+)', ' ', str(txt).lower()) + ' brk ' for txt in txtp])
    words = mystem.lemmatize(concated)
    sen = []
    for word in words:
      word = word.strip()
      if word != '\n' and word != '' and word not in russian_stopwords:
        if word == 'brk':
          result.append(' '.join(sen))
          sen = []
        else:
          sen.append(word)
  train[col] = pd.Series(result)
train.describe()

price  image_top_1  deal_probability  region_Алтайский край  \
count 1503424.0000 1503424.0000      1503424.0000           1503424.0000   
mean        7.8590    1148.9267            0.1391                 0.0276   
std         2.6887     989.0034            0.2601                 0.1639   
min         0.0000       0.0000            0.0000                 0.0000   
25%         6.2166     125.0000            0.0000                 0.0000   
50%         7.1317     960.0000            0.0000                 0.0000   
75%         8.8538    2078.0000            0.1509                 0.0000   
max        15.4249    3066.0000            1.0000                 1.0000   

       region_Башкортостан  region_Белгородская область  \
count         1503424.0000                 1503424.0000   
mean                0.0454                       0.0192   
std                 0.2082                       0.1372   
min                 0.0000                       0.0000   
25%                 0.0000                       0.0000   
50%                 0.0000                       0.0000   
75%                 0.0000                       0.0000   
max                 1.0000                       1.0000   

       region_Владимирская область  region_Волгоградская область  \
count                 1503424.0000                  1503424.0000   
mean                        0.0178                        0.0326   
std                         0.1322                        0.1776   
min                         0.0000                        0.0000   
25%                         0.0000                        0.0000   
50%                         0.0000                        0.0000   
75%                         0.0000                        0.0000   
max                         1.0000                        1.0000   

       region_Воронежская область  region_Иркутская область  \
count                1503424.0000              1503424.0000   
mean                       0.0293                    0.0293   
std                        0.1688                    0.1686   
min                        0.0000                    0.0000   
25%                        0.0000                    0.0000   
50%                        0.0000                    0.0000   
75%                        0.0000                    0.0000   
max                        1.0000                    1.0000   

       region_Калининградская область  region_Кемеровская область  \
count                    1503424.0000                1503424.0000   
mean                           0.0218                      0.0297   
std                            0.1460                      0.1697   
min                            0.0000                      0.0000   
25%                            0.0000                      0.0000   
50%                            0.0000                      0.0000   
75%                            0.0000                      0.0000   
max                            1.0000                      1.0000   

       region_Краснодарский край  region_Красноярский край  \
count               1503424.0000              1503424.0000   
mean                      0.0941                    0.0355   
std                       0.2919                    0.1852   
min                       0.0000                    0.0000   
25%                       0.0000                    0.0000   
50%                       0.0000                    0.0000   
75%                       0.0000                    0.0000   
max                       1.0000                    1.0000   

       region_Нижегородская область  region_Новосибирская область  \
count                  1503424.0000                  1503424.0000   
mean                         0.0490                        0.0416   
std                          0.2158                        0.1996   
min                          0.0000                        0.0000   
25%                          0.0000                        0.0000   
50%                 

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1503424 entries, 0 to 1503423
Columns: 1782 entries, item_id to std_parent_category_price
dtypes: float64(5), object(15), uint8(1762)
memory usage: 2.7+ GB


In [ ]:
train.to_csv(ROOT + 'simple_features.csv', index=False, header=True)